In [5]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys
from __future__ import division
#sys.path.append("/home/gias/dev/opinion/opinion/python/opinion")
sys.path.append(r"C:\dev\opinion\opinion\python\opinion")
import os
import utils.fileutils as fileutils
import utils.nlputils as nlputils
import cPickle as pickle
import pandas as pd
import numpy as np
import utils.metrics as metrics

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sentimentdetect.diversity.DiversityMetrics as dm

In [3]:
indir = r"C:\dev\opinion\papers\sentimentDiversity"
dirDataset = os.path.join(indir, "dataset")

In [16]:
def computeDiversity(infile, outfile):
    shannons = []
    simpsons = []
    invSimpsons = []
    df = fileutils.readExcel(infile, "Sheet1")
    for index, row in df.iterrows():
        sentence = row["sentence"]
        words = word_tokenize(sentence)
        data = dict()
        for w in words:
            #if w not in stopWords:
            data[w] = data.get(w, 0) + 1
        shannon = dm.shannon(data)
        shannons.append(shannon)
        simpson = dm.simpson(data)
        simpsons.append(simpson)
        simpson_inv = dm.inverse_simpson(data)        
        invSimpsons.append(simpson_inv)
    df["shannon"] = (pd.Series(shannons)).values
    df["simpson"]= (pd.Series(simpsons)).values
    df["simpson_inv"]= (pd.Series(invSimpsons)).values
    fileutils.writeExcel(outfile, "Sheet1", df)

In [18]:
infile = "ConsolidatedTSE.xls"
infile = os.path.join(dirDataset, infile)
computeDiversity(infile, infile)


In [25]:
infile = "DatasetLinAppReviewsWithToolOutputs.xls"
infile = os.path.join(dirDataset, infile)
computeDiversity(infile, infile)

In [26]:
infile = "DatasetLinJIRAWithToolOutputs.xls"
infile = os.path.join(dirDataset, infile)
computeDiversity(infile, infile)

In [28]:
infile = "DatasetLinStackOverflowWithToolOutputs.xls"
infile = os.path.join(dirDataset, infile)
computeDiversity(infile, infile)

In [37]:
infile = "DatasetSenti4SDWithToolOutputs.xls"
infile = os.path.join(dirDataset, infile)
computeDiversity(infile, infile)

In [14]:
def CheckClassifier(df, gt_name, clf_name):
    shannons = dict()
    simpsons = dict()
    inv_simpsons = dict()
    for t in [True, False]:
        shannons[t] = []
        simpsons[t] = []
        inv_simpsons[t]  = []
    for index, row in df.iterrows():
        #sentence = row[text_name]
        shannon = row["shannon"]
        simpson = row["simpson"]
        simpson_inv = row["simpson_inv"]
        manualLabel = row[gt_name]
        sentiLabel = row[clf_name]
        shannons[sentiLabel == manualLabel].append(shannon)
        simpsons[sentiLabel == manualLabel].append(simpson)
        inv_simpsons[sentiLabel == manualLabel].append(simpson_inv)
        #print manualLabel, sentiSELabel, manualLabel != sentiSELabel, shannon, simpson#, simpson_inv
    for t in [True, False]:
        print t
        sn = np.array(shannons[t])
        sm = np.array(simpsons[t])
        inv_sm = np.array(inv_simpsons[t])
        print "\t Shannon: Mean = %.3f. STD = %.3f. Median = %.3f"%(np.mean(sn), np.std(sn), np.median(sn))
        print "\t Simpson: Mean = %.3f. STD = %.3f. Median = %.3f"%(np.mean(sm), np.std(sm), np.median(sm))
        print "\t Simpson Inverse: Mean = %.3f. STD = %.3f. Median = %.3f"%(np.mean(inv_sm), np.std(inv_sm), np.median(inv_sm))   
    print "Changes between Truth and False."
    snT = np.array(shannons[True])
    mean_snT = np.mean(snT)
    smT = np.array(simpsons[True])
    mean_smT = np.mean(smT)
    inv_smT = np.array(inv_simpsons[True])
    mean_inv_smT = np.mean(inv_smT)
    
    snF = np.array(shannons[False])
    mean_snF = np.mean(snF)
    smF = np.array(simpsons[False])
    mean_smF = np.mean(smF)
    inv_smF = np.array(inv_simpsons[False])
    mean_inv_smF = np.mean(inv_smF)
    
    rsn = (mean_snF - mean_snT) *100 / mean_snT
    rsm = (mean_smF - mean_smT) *100 / mean_smT
    rsim = (mean_inv_smF - mean_inv_smT) *100 / mean_inv_smT
    print "Shannon = %.3f. Simson = %.3f. Inverse Simpson = %.3f"%(rsn, rsm, rsim)

In [19]:
infileOpiner = os.path.join(dirDataset, "ConsolidatedTSE.xls")
df = fileutils.readExcel(infileOpiner, "Sheet1")
df.head()

,thread,id,sentence,Sentistrength-se,SentiSE,InitialLabel,Sentistrength,Lexicons,OpinerDSO,ManualLabel,tid,sid,OpinerDSOSenti,Senti4SD,Stanford,SentiCR,shannon,simpson,simpson_inv
0,15936368,1,JAXB Bindings File Sets @XmlElement type to St...,1 -1,o,o,o,NaN,o,o,15936368:1,1,o,o,o,o,2.484907,0.083333,12.000000
1,15936368,2,I'm trying to create an CODETERM1 that takes i...,1 -1,o,o,o,NaN,o,o,15936368:2,2,o,o,o,o,2.566599,0.085938,11.636364
2,15936368,3,The purpose is simlply to remove timezone data...,1 -1,o,o,o,NaN,o,o,15936368:3,3,o,o,o,o,2.540036,0.081633,12.250000
3,15936368,4,It looks like this: CODESNIPPET_JAVA1,1 -1,o,p,p,NaN,o,o,15936368:4,4,p,o,o,o,1.791759,0.166667,6.000000
4,15936368,5,This works fine for the following code: CODESN...,3 -1,p,p,p,works,p,p,15936368:5,5,p,p,p,o,2.197225,0.111111,9.000000


In [20]:
gt_name = "ManualLabel"
clf_name = "Senti4SD"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 2.442. STD = 0.746. Median = 2.565
	 Simpson: Mean = 0.129. STD = 0.175. Median = 0.079
	 Simpson Inverse: Mean = 13.614. STD = 7.673. Median = 12.583
False
	 Shannon: Mean = 2.533. STD = 0.724. Median = 2.639
	 Simpson: Mean = 0.114. STD = 0.147. Median = 0.071
	 Simpson Inverse: Mean = 14.824. STD = 8.358. Median = 14.000
Changes between Truth and False.
Shannon = 3.712. Simson = -11.785. Inverse Simpson = 8.889


In [21]:
gt_name = "ManualLabel"
clf_name = "Sentistrength"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 2.434. STD = 0.742. Median = 2.565
	 Simpson: Mean = 0.129. STD = 0.171. Median = 0.081
	 Simpson Inverse: Mean = 13.537. STD = 7.618. Median = 12.349
False
	 Shannon: Mean = 2.548. STD = 0.725. Median = 2.639
	 Simpson: Mean = 0.113. STD = 0.149. Median = 0.071
	 Simpson Inverse: Mean = 15.004. STD = 8.444. Median = 14.000
Changes between Truth and False.
Shannon = 4.693. Simson = -12.518. Inverse Simpson = 10.835


In [22]:
gt_name = "ManualLabel"
clf_name = "SentiSE"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 2.426. STD = 0.766. Median = 2.565
	 Simpson: Mean = 0.134. STD = 0.182. Median = 0.080
	 Simpson Inverse: Mean = 13.539. STD = 7.776. Median = 12.565
False
	 Shannon: Mean = 2.580. STD = 0.677. Median = 2.686
	 Simpson: Mean = 0.104. STD = 0.123. Median = 0.071
	 Simpson Inverse: Mean = 15.233. STD = 8.324. Median = 14.000
Changes between Truth and False.
Shannon = 6.342. Simson = -22.470. Inverse Simpson = 12.510


In [29]:
infile = os.path.join(dirDataset, "DatasetLinAppReviewsWithToolOutputs.xls")
df = fileutils.readExcel(infile, "Sheet1")
df.head()

,sentence,ManualLabel,SentiSE,Senti4SD,shannon,simpson,simpson_inv
0,package file invalid i had my phone on factory...,n,n,n,3.798278,0.025531,39.168421
1,iffy nice clean app but sometimes it works and...,n,p,p,3.276098,0.041016,24.380952
2,cool just freezes everytime,n,p,p,1.386294,0.250000,4.000000
3,network error! suddenly after downloading an u...,n,p,n,3.352470,0.040123,24.923077
4,annoying it let me choose the pictures i want ...,n,n,n,2.714452,0.079365,12.600000


In [30]:
gt_name = "ManualLabel"
clf_name = "SentiSE"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 2.578. STD = 0.795. Median = 2.458
	 Simpson: Mean = 0.104. STD = 0.070. Median = 0.089
	 Simpson Inverse: Mean = 16.297. STD = 13.118. Median = 11.267
False
	 Shannon: Mean = 2.878. STD = 0.776. Median = 2.888
	 Simpson: Mean = 0.081. STD = 0.065. Median = 0.059
	 Simpson Inverse: Mean = 20.452. STD = 14.057. Median = 16.917
Changes between Truth and False.
Shannon = 11.613. Simson = -22.593. Inverse Simpson = 25.494


In [31]:
gt_name = "ManualLabel"
clf_name = "Senti4SD"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 2.584. STD = 0.792. Median = 2.482
	 Simpson: Mean = 0.105. STD = 0.072. Median = 0.087
	 Simpson Inverse: Mean = 16.304. STD = 13.194. Median = 11.516
False
	 Shannon: Mean = 2.909. STD = 0.774. Median = 2.926
	 Simpson: Mean = 0.077. STD = 0.059. Median = 0.058
	 Simpson Inverse: Mean = 21.015. STD = 13.963. Median = 17.308
Changes between Truth and False.
Shannon = 12.565. Simson = -26.323. Inverse Simpson = 28.897


In [39]:
infile = os.path.join(dirDataset, "DatasetSenti4SDWithToolOutputs.xls")
df = fileutils.readExcel(infile, "Sheet1") #pd.read_csv(infile, encoding='ISO-8859-1')
df.head()

,sentence,ManualLabel,SentiSE,shannon,simpson,simpson_inv
0,"Vineet, what you are trying to do is a terribl...",n,n,3.811141,0.024454,40.893617
1,"'Course I do, corrected.",p,o,1.791759,0.166667,6.000000
2,"Excellent, happy to help! If you don't mind, c...",p,p,2.736339,0.067901,14.727273
3,@DrabJay: excellent suggestion! Code changed. :-),p,p,2.369382,0.097222,10.285714
4,Any decent browser should protect against mali...,o,p,3.764228,0.025377,39.405405


In [40]:
gt_name = "ManualLabel"
clf_name = "SentiSE"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 2.889. STD = 0.790. Median = 2.896
	 Simpson: Mean = 0.080. STD = 0.071. Median = 0.057
	 Simpson Inverse: Mean = 21.321. STD = 13.800. Median = 17.640
False
	 Shannon: Mean = 3.225. STD = 0.717. Median = 3.389
	 Simpson: Mean = 0.057. STD = 0.057. Median = 0.037
	 Simpson Inverse: Mean = 27.383. STD = 14.130. Median = 27.272
Changes between Truth and False.
Shannon = 11.655. Simson = -28.766. Inverse Simpson = 28.435


In [41]:
infile = os.path.join(dirDataset, "DatasetLinJIRAWithToolOutputs.xls")
df = fileutils.readExcel(infile, "Sheet1") #pd.read_csv(infile, encoding='ISO-8859-1')
df.head()

,sentence,ManualLabel,SentiSE,Senti4SD,shannon,simpson,simpson_inv
0,guys... this is so stupid...,n,n,n,1.747868,0.183673,5.444444
1,I lost the whole morning cause HBase's RegionS...,n,o,n,3.467994,0.032407,30.857143
2,{quote}You are messing down deep below hbase i...,n,n,o,2.564949,0.076923,13.000000
3,And I think if we're going to do a sweep up of...,n,n,o,2.978508,0.052154,19.173913
4,"@idiot Yeah, I was on that idiot-path for a go...",n,n,p,3.262568,0.040000,25.000000


In [42]:
gt_name = "ManualLabel"
clf_name = "SentiSE"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 1.993. STD = 0.690. Median = 1.946
	 Simpson: Mean = 0.173. STD = 0.121. Median = 0.143
	 Simpson Inverse: Mean = 9.049. STD = 6.166. Median = 7.000
False
	 Shannon: Mean = 2.438. STD = 0.606. Median = 2.485
	 Simpson: Mean = 0.107. STD = 0.072. Median = 0.086
	 Simpson Inverse: Mean = 13.103. STD = 7.199. Median = 11.636
Changes between Truth and False.
Shannon = 22.320. Simson = -37.925. Inverse Simpson = 44.811


In [43]:
gt_name = "ManualLabel"
clf_name = "Senti4SD"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 1.968. STD = 0.685. Median = 1.946
	 Simpson: Mean = 0.175. STD = 0.116. Median = 0.143
	 Simpson Inverse: Mean = 8.865. STD = 6.334. Median = 7.000
False
	 Shannon: Mean = 2.261. STD = 0.678. Median = 2.303
	 Simpson: Mean = 0.135. STD = 0.110. Median = 0.100
	 Simpson Inverse: Mean = 11.438. STD = 6.731. Median = 10.000
Changes between Truth and False.
Shannon = 14.889. Simson = -22.786. Inverse Simpson = 29.020


In [44]:
infile = os.path.join(dirDataset, "DatasetLinStackOverflowWithToolOutputs.xls")
df = fileutils.readExcel(infile, "Sheet1") #pd.read_csv(infile, encoding='ISO-8859-1')
df.head()

,sentence,ManualLabel,SentiSE,Senti4SD,shannon,simpson,simpson_inv
0,But sadly this is not working.,n,n,n,1.945910,0.142857,7.000000
1,"So, everything builds fine, but when we try to...",n,p,p,3.044820,0.050296,19.882353
2,That is what is causing your null pointer exce...,n,o,o,2.163956,0.120000,8.333333
3,"All attempts I've made were, in a shortcut, un...",n,o,n,2.458311,0.088757,11.266667
4,Don't use.,n,o,o,1.386294,0.250000,4.000000


In [45]:
gt_name = "ManualLabel"
clf_name = "SentiSE"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 2.313. STD = 0.514. Median = 2.352
	 Simpson: Mean = 0.115. STD = 0.065. Median = 0.100
	 Simpson Inverse: Mean = 11.114. STD = 5.419. Median = 10.000
False
	 Shannon: Mean = 2.401. STD = 0.567. Median = 2.485
	 Simpson: Mean = 0.109. STD = 0.074. Median = 0.083
	 Simpson Inverse: Mean = 12.396. STD = 6.204. Median = 12.000
Changes between Truth and False.
Shannon = 3.790. Simson = -5.105. Inverse Simpson = 11.538


In [46]:
gt_name = "ManualLabel"
clf_name = "Senti4SD"
CheckClassifier(df, gt_name, clf_name)

True
	 Shannon: Mean = 2.316. STD = 0.511. Median = 2.352
	 Simpson: Mean = 0.115. STD = 0.065. Median = 0.100
	 Simpson Inverse: Mean = 11.132. STD = 5.423. Median = 10.000
False
	 Shannon: Mean = 2.388. STD = 0.576. Median = 2.485
	 Simpson: Mean = 0.111. STD = 0.074. Median = 0.085
	 Simpson Inverse: Mean = 12.289. STD = 6.180. Median = 11.786
Changes between Truth and False.
Shannon = 3.105. Simson = -3.372. Inverse Simpson = 10.395
